1. Web\mysite에서 python manage.py startapp myboard 실행(py36가상환경 하에서)

2. Web/mysite/blog에서 admin.py, forms.py, models.py, urls.py, views.py 등을 복사해서 Web/mysite/myboard 디렉토리에 붙여넣기 한다. (나중에 수정해서 쓰게)

3. mysite>templates에서 myboard 폴더를 만들고 base.html, detail.html, edit.html, list.html, login.html을 복사해서 붙여넣기 

4. mysite>mysite>settings.py 에서 INSTALLED_APPS에  'myboard'추가



5. mysite>mysite>urls.py에서 path 추가
    - path('myboard/', include('myboard.urls'))
    
    
6. mysite>myboard>models.py를 다음 셀과 같이 작성

In [ ]:
from django.db import models
from django.utils import timezone


# Create your models here.

class Board(models.Model):
    author = models.ForeignKey('auth.User', on_delete=models.CASCADE)  # 유저를 지울때 그 유저에 대한 정보를 다같이 지우라는 뜻
    title = models.CharField(max_length=200)
    text = models.TextField()  # 글자수에 제한 없는 텍스트
    created_date = models.DateTimeField(
        default=timezone.now)  # 날짜와 시간
    cnt=models.IntegerField(default=0)
    image = models.CharField(max_length=200, null=True, blank=True)
    category = models.CharField(max_length = 10, default='common')


    def __str__(self):
        return self.title

7. mysite>myboard>admins.py를 다음 셀과 같이 수정

In [ ]:
from django.contrib import admin
from . import models

# Register your models here.

admin.site.register(Board)

8. mysite>myboard>forms.py를 다음 셀과 같이 수정

In [ ]:
from django.forms import ValidationError
from django import forms
from . import models


def validator(value) :
    if len(value) < 5 : raise  ValidationError("길이가 너무 짧아요");

class BoardForm(forms.ModelForm):
    class Meta:
        model = models.Board
        fields = ['title', 'text', 'category']

    def __init__(self, *args, **kwargs):
        super(BoardForm, self).__init__(*args, **kwargs)
        self.fields['title'].validators = [validator]

9. mysite>myboard>urls.py를 다음 셀과 같이 수정

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('<int:pk>/<mode>/', views.BoardView.as_view(), name='myboard'),
]

10. mysite>myboard>views.py를 다음과 같이 수정한다.

In [ ]:
from django.shortcuts import render, get_object_or_404,redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth.models import User
from . import forms
from . import models

"""
def list(request) :
    username = request.session["username"]
    user = User.objects.get(username=username)
    data = models.Post.objects.all().filter(author=user)
    context = {"data":data, "username":username}
    return render(request, "blog/list.html", context)

def detail(request, pk) :
    p = get_object_or_404(models.Post, pk=pk)
    return render(request, "blog/detail.html", {"d":p})
"""


class BoardView(View) :
    def get(self, request, pk, mode):
        # get 요청
        # 0/list -> 리스트 출력
        # 5/detail -> 5번 post를 보여줘
        # 0/add -> 신규 데이터 작성 -> post 발생
        # 5/edit -> 5번 post 수정
        if mode == "add" :
            form = forms.BoardForm()
        elif mode == "list" :
            username = request.session["username"]
            user = User.objects.get(username=username)
            data = models.Board.objects.all().filter(author=user)
            context = {"data": data, "username": username}
            return render(request, "myboard/list.html", context)
        elif mode == "detail" :
            p = get_object_or_404(models.Board, pk=pk)
            p.cnt += 1
            p.save()
            return render(request, "myboard/detail.html", {"d": p})
        elif mode == "edit" :
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(instance = post)
        else :
            return HttpResponse("error page")

        return render(request, "myboard/edit.html", {"form":form})


    def post(self, request, pk, mode):
        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0:
            form = forms.BoardForm(request.POST)
        else:
            post = get_object_or_404(models.Board, pk=pk)
            form = forms.BoardForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
            post.save()
            return redirect("myboard", 0, 'list')
        return render(request, "myboard/edit.html", {"form": form})


class LoginView(View) :
    def get(self, request):
        return render(request, "myboard/login.html")

    def post(self, request):
        username = request.POST.get("username")
        password = request.POST.get("password")

        user = authenticate(username=username, password=password)
        if user == None :
            return redirect("login")
        request.session["username"] = username
        return redirect("list")

10. 수정 다 하다고 
    - python manage.py makemigrations
    - python manage.py migrate 실행

11. mysite>templates>myboard>edit.html을 다음과 같이 수정

In [ ]:
{% extends 'blog/base.html' %}

게시물 보기 <br>

{% block content %}


    {{d.title}}  <br>
    {{d.text|linebreaks}}    <!--'|linebreaks' : enter를 인식 못해서 이걸 자동적으로 <br>로 바꿔줌-->

    <a href="{% url 'myboard' d.pk 'edit' %}">수정</a>
{% endblock %}



12. mysite>templates>myboard>list.html를 다음과 같이 수정

In [ ]:
{% extends 'myboard/base.html' %}   <!-- 파이썬에서 import 하는 개념이라고 생각하면 됌-->

{% block content %}
<a href = "{% url 'myboard' 0 'add'%}">Add</a> <br>
{% for d in data %}
<a href="{% url 'myboard' d.pk 'detail' %}"> {{d.title}} </a> <br>
{% endfor%}

{% endblock %}

